In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
os.environ['THEANO_FLAGS'] = 'device=gpu'
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from lasagnekit.datasets.mnist import MNIST
from lasagnekit.misc.plot_weights import dispims_color
import theano.tensor as T
import theano

In [ ]:
data = MNIST()
data.load()
nb = 10000
Y = data.X[0:nb]

#X = np.arange(len(Y), dtype='int32')
X = np.random.uniform(-1, 1, size=(nb, z_dim))
Z = data.y[0:nb]

In [ ]:
z_dim = 100
nb_hidden = [
    100, 256, 512, 784
]
hid = []
nb_examples = X.shape[0]

#x = Input(shape=(1,), dtype='int32')
#inp = x
#x = Embedding(input_dim=nb_examples, output_dim=z_dim)(x)
#x = Flatten()(x)


x = Input(shape=(z_dim,), dtype='float32')
inp = x
for i, h in enumerate(nb_hidden):
    x = Dense(h)(x)
    if i == len(nb_hidden) - 1:
        x = BatchNormalization()(x)
        s = 'sigmoid'
    else:
        s = 'relu'
    x = Activation(s)(x)
    hid.append(x)
outp = x
model = Model(inp, outp)
optimizer = Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
for l in model.layers:
    print(l)

In [ ]:
encode = theano.function([model.layers[0].input], model.layers[6].input)

In [ ]:
epoch = 0
nb_per = 10

In [ ]:
model.fit(X, Y, verbose=1, nb_epoch=nb_per)
epoch += nb_per
print(epoch)

In [ ]:
y_pred = model.predict(X[0:100])
y_pred = y_pred.reshape((y_pred.shape[0], 28, 28, 1))
y_pred = y_pred * np.ones((1, 1, 1, 3))
img = dispims_color(y_pred)
fig = plt.figure(figsize=(20, 20))
plt.imshow(img, interpolation='none')

In [ ]:
from sklearn.manifold import TSNE
w = TSNE().fit_transform(encode(X.astype(np.float32))[0:200])

In [ ]:
#fig = plt.figure(figsize=(20, 20))
fig = plt.figure()
plt.scatter(w[:, 0], w[:, 1], c=Z[0:200])#, marker='+')

In [ ]:
w = model.layers[1].W.get_value()
c=plt.imshow(np.corrcoef(w.T))
plt.colorbar()

In [ ]:
fw = model.layers[-3].W.get_value()
fw = fw.reshape((fw.shape[0], 28, 28, 1))
fw = fw * np.ones((1, 1, 1, 3))

In [ ]:
img = dispims_color(fw)
fig = plt.figure(figsize=(20, 20))
plt.imshow(img, cmap='gray')

In [ ]:
from scipy.stats import multivariate_normal
print(model.layers[9])
l=model.layers[9]

In [ ]:
xx = T.matrix()
c = theano.function([model.layers[3].ones], model.layers[8].input)
h = np.random.multivariate_normal(w.mean(axis=0), np.cov(w.T), size=128).astype(np.float32)
h = w[0:100]
pred = c(h)
pred = pred.reshape((pred.shape[0], 28, 28, 1))
pred = pred * np.ones((1, 1, 1, 3))

In [ ]:
w = model.layers[1].W.get_value()
img = dispims_color(pred)
fig = plt.figure(figsize=(20, 20))
plt.imshow(img, cmap='gray')